## Вариант 1 - Простая логика

In [1]:
from tqdm import tqdm, tqdm_notebook

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import pandas_profiling

import seaborn as sns 

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve

import sys
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

from datetime import datetime as datetime


In [2]:
# Читаем данные
_df = pd.read_csv('data/aac_shelter_outcomes.csv', dtype={'name':object})
_df.head(10)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male
6,1 year,A693700,Other,Squirrel Mix,Tan,2013-12-13T00:00:00,2014-12-13T12:20:00,2014-12-13T12:20:00,NaN,Suffering,Euthanasia,Unknown
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female


 - age_upon_outcome => The age of the animal upon outcome
 - animal_id => Unique ID given to each animal upon intake
 - animal_type => The type of animal, such as 'dog', 'cat', 'bird', etc.
 - breed => The breed of the animal
 - color => Color of the animal
 - date_of_birth => Date of birth of the animal (estimated if exact birthdate is not known)
 - datetime => The date and time the outcome occurred
 - monthyear => Month and year of the outcome as datetime
 - name => The name of the animal
 - outcome_subtype => A more specific description of the outcome type where appropriate
 - outcome_type => The type of outcome. Please see the description for definitions of the outcomes if needed
 - sex_upon_out => comeColumn denoting the gender of the animal and if it has been spayed or neutered upon outcome

- age_upon_outcome => Возраст животного на исходе
- animal_id => Уникальный идентификатор, присвоенный каждому животному при приеме внутрь
- animal_type => Тип животного, такой как «собака», «кошка», «птица» и т. д.
- порода => порода животных
- цвет => окрас животного
- date_of_birth => Дата рождения животного (оценивается, если точная дата рождения неизвестна)
- datetime => Дата и время, когда произошел результат
- monthyear => Месяц и год результата как datetime
- name => Название животного
- outcome_subtype => Более конкретное описание типа результата, где это уместно
- outcome_type => Тип результата. Пожалуйста, смотрите описание для определения результатов, если это необходимо
- sex_upon_outcome => обозначает пол животного и если оно было стерилизовано или кастрировано по результатам

In [3]:
# Создаем копию датасета, чтобы при тестах не перечитывать файл заново
df = _df.copy(deep=True)

In [4]:
# Мы видим, что столбцы 'datetime' и 'monthyear' совпадают. Удалим один из них
df.drop(['monthyear'], axis=1, inplace=True)
df.head(15)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03T00:00:00,2013-10-07T13:06:00,*Edgar,Partner,Transfer,Intact Male
6,1 year,A693700,Other,Squirrel Mix,Tan,2013-12-13T00:00:00,2014-12-13T12:20:00,NaN,Suffering,Euthanasia,Unknown
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23T00:00:00,2014-12-08T15:55:00,*Ella,Partner,Transfer,Spayed Female
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female


In [5]:
# Для удобства работы далее переведем 'date_of_birth' в тип datetime
df['date_of_birth'] = df['date_of_birth'].progress_apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))

In [6]:
# Для удобства работы далее переведем 'datetime' в тип datetime
df['datetime'] = df['datetime'].progress_apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))

In [7]:
# Мы видим, что некоторые имена ('name') со звездочками. Создадим дополнительный признак (на всякий). Потом "поиграемся" с ним.
df['name'] = df['name'].fillna('')
df['is_star_name'] = df['name'].progress_apply(lambda x: True if '*' in x else False)
df.head(15)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,name,outcome_subtype,outcome_type,sex_upon_outcome,is_star_name
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,,Partner,Transfer,Intact Male,False
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,Lucy,Partner,Transfer,Spayed Female,False
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,*Johnny,NaN,Adoption,Neutered Male,True
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,Monday,Partner,Transfer,Neutered Male,False
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07,2014-07-07 14:04:00,,Rabies Risk,Euthanasia,Unknown,False
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03,2013-10-07 13:06:00,*Edgar,Partner,Transfer,Intact Male,True
6,1 year,A693700,Other,Squirrel Mix,Tan,2013-12-13,2014-12-13 12:20:00,,Suffering,Euthanasia,Unknown,False
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23,2014-12-08 15:55:00,*Ella,Partner,Transfer,Spayed Female,True
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16,2014-08-14 18:45:00,Lucy,NaN,Adoption,Intact Female,False
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26,2014-06-29 17:45:00,*Frida,Offsite,Adoption,Spayed Female,True


In [9]:
# Сделаем профайлинг :)
df.profile_report()

### Мысли вслух после профайлинга
Если просто порассуждать, то мне кажется:
 - Имена никак не должны влиять на предсказываемое значение. Единственное, что я проверил бы как влияет наличие/отсутствие имени (есть ли какая-то корреляция?);
 - animal_id (по сути ID животного) тоже никак не должно влиять на модель (будет только мешать). Я бы удалил этот признак;
 - из колонки `sex_upon_outcome` мы можем вытащить пол животного. Признак будет не совсем полноценным, т.к. о многих животных мы не имеем этой информации, но для эксперимента стоит это сделать.
 -  из той же колонки `sex_upon_outcome` я бы сгенерировал новый признак "Воздействие", который показывал бы было ли произведено с животным какое-то действие? Подразумевая кастрацию/стерелизацию.

In [10]:
# Из профайлинга видно, что animal_id повторяются. Я их отфильтрую, этот параметр (по-сути) id животного.
df = df.loc[df.duplicated(['animal_id']) == False]
len(df)

70855

In [11]:
# outcome_subtype has 36369 (51.3%) missing values
# Очень много значени отсутствует. Для уменьшения размерности модели можно (в качестве эксперимента) удалить этот признак.

df.drop(['outcome_subtype'], axis=1, inplace=True)
len(df)

70855

In [12]:
# Добавим признак наличия имени
df['has_name'] = df['name'].progress_apply(lambda x: True if x != '' else False)
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,name,outcome_type,sex_upon_outcome,is_star_name,has_name
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,,Transfer,Intact Male,False,False
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,Lucy,Transfer,Spayed Female,False,True
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,*Johnny,Adoption,Neutered Male,True,True
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,Monday,Transfer,Neutered Male,False,True
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07,2014-07-07 14:04:00,,Euthanasia,Unknown,False,False


In [13]:
# Удалим имена
df.drop(['name'], axis=1, inplace=True)

In [14]:
# Удалим animal_id
df.drop(['animal_id'], axis=1, inplace=True)

In [15]:
df.head()

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,is_star_name,has_name
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,Transfer,Intact Male,False,False
1,1 year,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,Transfer,Spayed Female,False,True
2,1 year,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,Adoption,Neutered Male,True,True
3,9 years,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,Transfer,Neutered Male,False,True
4,5 months,Other,Bat Mix,Brown,2014-01-07,2014-07-07 14:04:00,Euthanasia,Unknown,False,False


In [16]:
# Сделаем СНОВА профайлинг :)
df.profile_report()



breed has a high cardinality: `2128` distinct values	
color has a high cardinality: `525` distinct values


Очень много уникальныйх значений в Породе (breed) и Цвет (color)

Еще я бы проработал `age_upon_outcome`

In [17]:
# Обработаем 'age_upon_outcome_in_weeks' таким образом, чтобы нормализовать данные.
# Я предпочитаю приведедение к одной размерности: возьму и все даты переведу в недели, 
# т.к. это самое низкое по рангу деление в нашем df

def get_weeks_from_age_upon_outcome(x):
    y = {
        'weeks': 0,
        'year': 48,
        'month': 4
    }
    
    if type(x) == str:
        digital = x.split(' ')[0]
        return digital
    else:
        return '0'
    

df['age_upon_outcome_in_weeks'] = df['age_upon_outcome'].progress_apply(lambda x: get_weeks_from_age_upon_outcome(x))


In [18]:
# Просто для контроля
df.groupby('age_upon_outcome_in_weeks').count().reset_index()[ ['age_upon_outcome_in_weeks', 'age_upon_outcome'] ]


,age_upon_outcome_in_weeks,age_upon_outcome
0,0,93
1,1,17082
2,10,2099
3,11,762
4,12,563
5,13,354
6,14,237
7,15,192
8,16,96
9,17,54


In [19]:
# Убиваем исходный столбец 
df.drop('age_upon_outcome', axis=1, inplace=True)
df.head()

,animal_type,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,is_star_name,has_name,age_upon_outcome_in_weeks
0,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,Transfer,Intact Male,False,False,2
1,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,Transfer,Spayed Female,False,True,1
2,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,Adoption,Neutered Male,True,True,1
3,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,Transfer,Neutered Male,False,True,9
4,Other,Bat Mix,Brown,2014-01-07,2014-07-07 14:04:00,Euthanasia,Unknown,False,False,5


In [20]:
# Смотрим количество NaN
df.isna().sum()

animal_type                  0
breed                        0
color                        0
date_of_birth                0
datetime                     0
outcome_type                 8
sex_upon_outcome             2
is_star_name                 0
has_name                     0
age_upon_outcome_in_weeks    0
dtype: int64

In [21]:
# Убираем все Nan
df = df.dropna() 
df.isna().sum()

animal_type                  0
breed                        0
color                        0
date_of_birth                0
datetime                     0
outcome_type                 0
sex_upon_outcome             0
is_star_name                 0
has_name                     0
age_upon_outcome_in_weeks    0
dtype: int64

In [23]:
df.isna().sum()

animal_type                  0
breed                        0
color                        0
date_of_birth                0
datetime                     0
outcome_type                 0
sex_upon_outcome             0
is_star_name                 0
has_name                     0
age_upon_outcome_in_weeks    0
dtype: int64

In [24]:
%%time

# Переведем animal_type в one-hot-encoding
df['animal_type'].unique()

types = set(df['animal_type'].unique())
animal_type = pd.DataFrame(columns=types)


def name_to_vector(x, types, result_df):
    # Временный словарь
    _dic = dict.fromkeys(types)
    
    # Бежим по строкам df
    for index, row in x.iterrows():
        _dic = dict.fromkeys(types)
        
        # перебираем наши значения
        for tps in types:
            if tps in x['animal_type'][index]:
                _dic.update({ f'{tps}' : 1})
            
            else:
                _dic.update({ f'{tps}' : 0})
        
        # Добавляем строку в рузельтирующий df
        result_df = result_df.append(_dic, ignore_index=True)

    #Возвращаем результат
    return result_df


a = name_to_vector(df, types, animal_type)
a.to_clipboard()
a.head()




Wall time: 2min 41s


,Other,Livestock,Dog,Cat,Bird
0,0,0,0,1,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,1,0,0,0,0


In [25]:
a.shape

(70846, 5)

In [26]:
df.shape

(70846, 10)

In [27]:
df1 = df.copy(deep=True).reset_index()
df2 = a.copy(deep=True).reset_index()

df = pd.concat([df1, df2], axis=1)
df.shape
df.isnull().sum()

index                        0
animal_type                  0
breed                        0
color                        0
date_of_birth                0
datetime                     0
outcome_type                 0
sex_upon_outcome             0
is_star_name                 0
has_name                     0
age_upon_outcome_in_weeks    0
index                        0
Other                        0
Livestock                    0
Dog                          0
Cat                          0
Bird                         0
dtype: int64

In [29]:
# Убиваем исходный столбец animal_type
df.drop('animal_type', axis=1, inplace=True)
df.head()

,index,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,is_star_name,has_name,age_upon_outcome_in_weeks,index,Other,Livestock,Dog,Cat,Bird
0,0,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,Transfer,Intact Male,False,False,2,0,0,0,0,1,0
1,1,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,Transfer,Spayed Female,False,True,1,1,0,0,1,0,0
2,2,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,Adoption,Neutered Male,True,True,1,2,0,0,1,0,0
3,3,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,Transfer,Neutered Male,False,True,9,3,0,0,1,0,0
4,4,Bat Mix,Brown,2014-01-07,2014-07-07 14:04:00,Euthanasia,Unknown,False,False,5,4,1,0,0,0,0


In [30]:
df.head()

,index,breed,color,date_of_birth,datetime,outcome_type,sex_upon_outcome,is_star_name,has_name,age_upon_outcome_in_weeks,index,Other,Livestock,Dog,Cat,Bird
0,0,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,Transfer,Intact Male,False,False,2,0,0,0,0,1,0
1,1,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,Transfer,Spayed Female,False,True,1,1,0,0,1,0,0
2,2,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,Adoption,Neutered Male,True,True,1,2,0,0,1,0,0
3,3,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,Transfer,Neutered Male,False,True,9,3,0,0,1,0,0
4,4,Bat Mix,Brown,2014-01-07,2014-07-07 14:04:00,Euthanasia,Unknown,False,False,5,4,1,0,0,0,0


In [33]:
# Преобразуем str в int
df['age_upon_outcome_in_weeks'] = df['age_upon_outcome_in_weeks'].astype('int64')

In [34]:
# Преобразуем target в int

outcome_type = df['outcome_type'].unique()
outcome_type_dict = {}

for i,x in enumerate(outcome_type):
    outcome_type_dict.update({x : i})
    

def outcome_type_to_int(x, outcome_type_dict):
    for key in outcome_type_dict.keys():
        if key == x:
            return outcome_type_dict[key]
    return -1
        
df['outcome_type'] = df['outcome_type'].progress_apply(lambda x: outcome_type_to_int(x, outcome_type_dict))

In [36]:
# X = df[ ['animal_type', 'breed', 'color', 'date_of_birth', 'datetime', 'sex_upon_outcome', 'is_star_name', 'has_name'] ]
X = df[ ['is_star_name', 'has_name', 'age_upon_outcome_in_weeks', 'Other', 'Livestock', 'Bird', 'Cat', 'Dog'] ]

In [37]:
y = df['outcome_type']

In [38]:
X.shape

(70846, 8)

In [39]:
y.shape

(70846,)

In [42]:
X.head()

,is_star_name,has_name,age_upon_outcome_in_weeks,Other,Livestock,Bird,Cat,Dog
0,False,False,2,0,0,0,1,0
1,False,True,1,0,0,0,0,1
2,True,True,1,0,0,0,0,1
3,False,True,9,0,0,0,0,1
4,False,False,5,1,0,0,0,0


In [43]:
y.head()


0    0
1    0
2    1
3    0
4    2
Name: outcome_type, dtype: int64

In [85]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


# Создаем тестовую и контрольную выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 21)



In [86]:
# Обучаем модель. Обязательно делаем n_jobs=-1, чтобы загрузить работой все доступные ядра процессора
model = RandomForestClassifier(n_estimators=500, max_depth=30, n_jobs=-1, bootstrap=True, random_state=21).fit(train, train_labels)

# Смотрим величину ошибки для выборок
err_train = np.mean(y_train != model.predict(X_train))
err_test  = np.mean(y_test  != model.predict(X_test))

print(err_train,err_test)

0.40273268780171073 0.40815289501171553


Как-то слишком много)

In [87]:
# И теперь давайте посмотрим на значимость параметров
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

feature_names = X.columns

print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5s}' ({:.4f})".format(f + 1, feature_names[idx], importances[idx]))

Feature importances:
 1. feature 'has_name' (0.3276)
 2. feature 'is_star_name' (0.1716)
 3. feature 'Other' (0.1663)
 4. feature 'age_upon_outcome_in_weeks' (0.1162)
 5. feature 'Cat  ' (0.1091)
 6. feature 'Dog  ' (0.1060)
 7. feature 'Bird ' (0.0031)
 8. feature 'Livestock' (0.0002)


Видимо, я зря вытащил типы животных в отдельные признаки :)


In [88]:

# Метод N ближайших соседей
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

y_train_predict = knn.predict(X_train)
y_test_predict = knn.predict(X_test)

err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test  != y_test_predict)
print (err_train, err_test)

0.43539508229116675 0.43909324450215964


Слишком большая ошибка

In [89]:
# градиентный бустинг деревьев решений¶
from sklearn import ensemble
gbt = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=21)
gbt.fit(X_train, y_train)

err_train = np.mean(y_train != gbt.predict(X_train))
err_test = np.mean(y_test != gbt.predict(X_test))
print (err_train, err_test)

0.4030432205064506 0.4093385653389041


Бустинг работает примерно так же, как и Случайный лес